In [9]:
import pandas as pd
import numpy as np
import json

In [10]:
rec_df = pd.read_csv('recommendation_resources/preprocessed_lyrics.csv', index_col=0)
rec_df.head()

,artist,song_name,file_name,lyrics,clean_lyrics
0,Gary Wright,Love Is Alive,love_is_alive.txt,"Well, I think it's time to get ready To realiz...","['well', 'think', 'time', 'get', 'ready', 'rea..."
1,LMFAO,Party Rock Anthem,party_rock_anthem.txt,Party rock Let's go! Party rock is in the hou...,"['party', 'rock', 'let', 'us', 'go', 'party', ..."
2,Metallica,2 X 4,2_x_4.txt,"I’m gonna make you, shake you, take you I’m go...","['go', 'make', 'shake', 'take', 'go', 'one', '..."
3,Metallica,Prince Charming,prince_charming.txt,There’s a black cloud overhead That’s me And t...,"['black', 'cloud', 'overhead', 'poison', 'ivy'..."
4,Metallica,Now That We'Re Dead,now_that_we're_dead.txt,When darkness falls May it be That we should s...,"['darkness', 'fall', 'may', 'see', 'light', 'r..."


# Generate Sample Recommendations

In [11]:
all_recs = []

for i in rec_df.index:
    rec_ids = [np.random.randint(0, len(rec_df)) for i in range(5)]
    recs = {}
    for j in rec_ids:
        recs[rec_df.artist[j]]={rec_df.song_name[j]:rec_df.file_name[j]}
    all_recs.append(recs)

rec_df['recommendations'] = all_recs
rec_df.head()

,artist,song_name,file_name,lyrics,clean_lyrics,recommendations
0,Gary Wright,Love Is Alive,love_is_alive.txt,"Well, I think it's time to get ready To realiz...","['well', 'think', 'time', 'get', 'ready', 'rea...",{'Bryan Adams': {'(Everything I Do) I Do It Fo...
1,LMFAO,Party Rock Anthem,party_rock_anthem.txt,Party rock Let's go! Party rock is in the hou...,"['party', 'rock', 'let', 'us', 'go', 'party', ...","{'Anthrax': {'King Size': 'king_size.txt'}, 'C..."
2,Metallica,2 X 4,2_x_4.txt,"I’m gonna make you, shake you, take you I’m go...","['go', 'make', 'shake', 'take', 'go', 'one', '...",{'Billy Joel': {'And So It Goes': 'and_so_it_g...
3,Metallica,Prince Charming,prince_charming.txt,There’s a black cloud overhead That’s me And t...,"['black', 'cloud', 'overhead', 'poison', 'ivy'...",{'Smokey Robinson & The Miracles': {'Baby Baby...
4,Metallica,Now That We'Re Dead,now_that_we're_dead.txt,When darkness falls May it be That we should s...,"['darkness', 'fall', 'may', 'see', 'light', 'r...",{'Jim Reeves': {'He'Ll Have To Go': 'he'll_hav...


### Create JSON File with Recommendations

In [12]:
recs = {}
for artist in rec_df.artist.unique():
    recs[artist] = {}

for i in rec_df.index:
    song_info = {
        'file_name':rec_df.file_name[i],
        'recommendations':rec_df.recommendations[i],
    }
    recs[rec_df.artist[i]][rec_df.song_name[i]]=song_info

with open("recommendations.json", "w") as outfile:  
    json.dump(recs, outfile)

recs

{'Gary Wright': {'Love Is Alive': {'file_name': 'love_is_alive.txt',
   'recommendations': {'Bryan Adams': {'(Everything I Do) I Do It For You': '(everything_i_do)_i_do_it_for_you.txt'},
    'Righteous Brothers': {"I'M Leaving It Up To You": "i'm_leaving_it_up_to_you.txt"},
    'Rita Coolidge': {'(Your Love Has Lifted Me) Higher And Higher': '(your_love_has_lifted_me)_higher_and_higher.txt'},
    'Supremes': {"You Can'T Do That": "you_can't_do_that.txt"},
    'Frankie Valli': {'My Eyes Adored You': 'my_eyes_adored_you.txt'}}}},
 'LMFAO': {'Party Rock Anthem': {'file_name': 'party_rock_anthem.txt',
   'recommendations': {'Anthrax': {'King Size': 'king_size.txt'},
    'Carpenters': {'(They Long To Be) Close To You': '(they_long_to_be)_close_to_you.txt'},
    'Pearl Jam': {'Hitchhiker': 'hitchhiker.txt'},
    'Aerosmith': {'Critical Mass': 'critical_mass.txt'},
    'Bee Gees': {'Night Fever': 'night_fever.txt'}}}},
 'Metallica': {'2 X 4': {'file_name': '2_x_4.txt',
   'recommendations': {